# Scraping details from Indeed job posts

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm # tool to track progress

In [2]:
df = pd.read_csv('../Data/indeed_remote_jobs.csv')
df.head()

,title,company,location,job_link
0,Work From Home 100% Virtual - Customer Service...,Amazon Support Services Costa Rica SRL,Remote,https://www.indeed.com/viewjob?jk=1d00f748ae6f...
1,Recruiting Assistant,Northrop Grumman,"Falls Church, VA 22042",https://www.indeed.com/viewjob?jk=28646ed1f674...
2,Healthcare Customer Service Representative,Sutherland,"Clifton, NJ 07015",https://www.indeed.com/viewjob?jk=deda26e36053...
3,Marketing Researcher (100% Remote),The Mom Project,Remote,https://www.indeed.com/viewjob?jk=6eea105877ec...
4,Geylang/Jalan Besar Babysitting/Cleaning Job,MEIDE Singapore,Remote,https://www.indeed.com/viewjob?jk=e849860c73fc...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6056 entries, 0 to 6055
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     6056 non-null   object
 1   company   6056 non-null   object
 2   location  6056 non-null   object
 3   job_link  6056 non-null   object
dtypes: object(4)
memory usage: 189.4+ KB


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(1481, 4)

In [7]:
detail_list=[]
for url in tqdm(df.job_link):
    chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
#     try:
#         driver.find_element(By.XPATH, "//*[contains(text(), 'This job has expired on Indeed')]")
#         expire = True
#     except:
#         expire = False
    try:
        desc = driver.find_element(By.ID, "jobDescriptionText")
        desc = desc.text.split("\n")
    except:
        desc = np.nan
    try:
        detail = driver.find_element(By.ID, "jobDetailsSection")
        detail = detail.text.split("\n")
    except:
        detail = np.nan

    detail_dict = {"job_link": url, "detail": detail, "description": desc}
    detail_list.append(detail_dict)
    driver.quit()
    time.sleep(3)


In [8]:
for url in tqdm(df.job_link[857:]):
    chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
#     try:
#         driver.find_element(By.XPATH, "//*[contains(text(), 'This job has expired on Indeed')]")
#         expire = True
#     except:
#         expire = False
    try:
        desc = driver.find_element(By.ID, "jobDescriptionText")
        desc = desc.text.split("\n")
    except:
        desc = np.nan
    try:
        detail = driver.find_element(By.ID, "jobDetailsSection")
        detail = detail.text.split("\n")
    except:
        detail = np.nan

    detail_dict = {"job_link": url, "detail": detail, "description": desc}
    detail_list.append(detail_dict)
    driver.quit()
    time.sleep(3)

100%|██████████| 624/624 [1:34:50<00:00,  9.12s/it]   


In [9]:
len(detail_list)

1481

In [10]:
detail_df = pd.DataFrame(detail_list)
print(detail_df.shape)
detail_df.head()

(1481, 3)


,job_link,detail,description
0,https://www.indeed.com/viewjob?jk=1d00f748ae6f...,"[Job details, Job Type, Full-time]","[Basic Requirements:, Located at Costa Rica., ..."
1,https://www.indeed.com/viewjob?jk=28646ed1f674...,"[Job details, Salary, $46,100 - $76,800 a year...","[Requisition ID: R10030138, Category: Human Re..."
2,https://www.indeed.com/viewjob?jk=deda26e36053...,NaN,"[Job Description:, , Sutherland is seeking ent..."
3,https://www.indeed.com/viewjob?jk=6eea105877ec...,"[Job details, Job Type, Full-time]",[Our Customer is a business and financial soft...
4,https://www.indeed.com/viewjob?jk=e849860c73fc...,NaN,NaN


In [11]:
detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job_link     1481 non-null   object
 1   detail       1289 non-null   object
 2   description  1477 non-null   object
dtypes: object(3)
memory usage: 34.8+ KB


In [12]:
final_df = pd.merge(df, detail_df, on= ['job_link'])

In [13]:
final_df

,title,company,location,job_link,detail,description
0,Work From Home 100% Virtual - Customer Service...,Amazon Support Services Costa Rica SRL,Remote,https://www.indeed.com/viewjob?jk=1d00f748ae6f...,"[Job details, Job Type, Full-time]","[Basic Requirements:, Located at Costa Rica., ..."
1,Recruiting Assistant,Northrop Grumman,"Falls Church, VA 22042",https://www.indeed.com/viewjob?jk=28646ed1f674...,"[Job details, Salary, $46,100 - $76,800 a year...","[Requisition ID: R10030138, Category: Human Re..."
2,Healthcare Customer Service Representative,Sutherland,"Clifton, NJ 07015",https://www.indeed.com/viewjob?jk=deda26e36053...,NaN,"[Job Description:, , Sutherland is seeking ent..."
3,Marketing Researcher (100% Remote),The Mom Project,Remote,https://www.indeed.com/viewjob?jk=6eea105877ec...,"[Job details, Job Type, Full-time]",[Our Customer is a business and financial soft...
4,Geylang/Jalan Besar Babysitting/Cleaning Job,MEIDE Singapore,Remote,https://www.indeed.com/viewjob?jk=e849860c73fc...,NaN,NaN
...,...,...,...,...,...,...
1476,Biostatistician,Actalent,"Philadelphia, PA 19019",https://www.indeed.com/viewjob?jk=9133afd848d1...,"[Job details, Salary, $57.50 an hour, Job Type...","[Equivalent Experience, , Description:, Job Su..."
1477,Remote After Training CSR,Aston Carter,"Portland, OR 97086",https://www.indeed.com/viewjob?jk=2e85fc7c9988...,"[Job details, Salary, $35,360 a year, Job Type...","[Equivalent Experience, , Now hiring for a Cus..."
1478,Loan Officer Assistant (Remote),Aston Carter,"Charleston, SC 29401",https://www.indeed.com/viewjob?jk=0fdd1d55ab5e...,"[Job details, Salary, $21 an hour, Job Type, C...","[Equivalent Experience, , I'm currently suppor..."
1479,100% Remote - Sitecore Developer,CyberCoders,"New York, NY 10007",https://www.indeed.com/viewjob?jk=3da4e208f1b4...,"[Job details, Salary, $100,000 - $130,000 a ye...","[100% Remote - Sitecore Developer, We are a le..."


In [14]:
final_df.to_csv('../Data/indeed_remote_jobs_detail.csv',index=False)

In [15]:
#the details and text will be cleaned in a separate notebook